In [8]:
from linearmodels import PanelOLS
import linearmodels as LM
#from linearmodels.panel import PanelData
import pandas as pd
import numpy as np

# 生成一个新的示例数据集
np.random.seed(0)  # 为了可重现性设置随机种子

# 定义样本大小和分组
n_entities = 50  # 实体数量
n_periods = 2  # 时间段数量（例如，Before 和 After）
n_samples = n_entities * n_periods  # 总样本量

# 生成模拟数据
entities = np.repeat(np.arange(n_entities), n_periods)
time_periods = np.tile(np.array([0, 1]), n_entities)  # 0 代表 Before，1 代表 After
treatment = np.random.choice([0, 1], size=n_entities, replace=True)  # 随机分配治疗组
treatment_expanded = np.repeat(treatment, n_periods)

# 假设处理效果增加2到结果变量
outcome = np.random.normal(5, 1, size=n_samples) + treatment_expanded * time_periods * 2

# 构建 DataFrame
df = pd.DataFrame({
    'entity': entities,
    'time': time_periods,
    'treatment': treatment_expanded,
    'outcome': outcome
})

df.head()


# 假设 df 是你的 DataFrame
# 以下代码假设 df 已经正确设置，包含以下列：
# 'entity' - 分组标识
# 'time' - 时间标识，比如前后
# 'treatment' - 是否接受处理的二元变量
# 'outcome' - 结果变量

# 准备面板数据
df['entity'] = pd.Categorical(df['entity'])
df['time'] = pd.Categorical(df['time'])
panel_data = LM.panel(df.set_index(['entity', 'time']))

# 定义模型
# 这里 'EntityEffects' 表示加入了实体（固定）效应
# 'TimeEffects' 可以选择性加入，如果你想控制不随实体变化但随时间变化的效应
mod = PanelOLS.from_formula('outcome ~ treatment * time + EntityEffects', data=panel_data)

# 拟合模型，使用群聚标准误
# 'cluster_entity=True' 表示按照实体群聚标准误
# 如果你有多个维度需要群聚（比如时间和实体），可以使用 cluster_time=True 或者提供一个 cluster_id
res = mod.fit(cov_type='clustered', cluster_entity=True)

# 输出结果
print(res)

TypeError: 'module' object is not callable